# 8-Puzzle (BFS/DFS) + Pygame GUI (single-notebook version)

This notebook is designed to run in **Cursor / VS Code notebooks**.

## What you get
- `PuzzleState` with `board`, `parent`, `move`, `depth`, plus `successor()`
- `AgentSolver` implementing **BFS** and **DFS** and path reconstruction
- A modified **Pygame GUI** that can **solve and animate** the solution

## Controls (when the GUI window opens)
- Arrow keys: move manually
- **B**: BFS solve + animate
- **D**: DFS solve + animate
- **R**: reshuffle solvable puzzle
- **G**: set goal state


## 0) Install dependencies (run once)

If `pygame` or `numpy` is missing, run the next cell.


In [1]:
# If you're using a virtual environment, make sure it is activated before running this.
# In many notebook setups, this works directly:

import sys, subprocess

def pip_install(pkgs):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *pkgs])

try:
    import numpy  # noqa
    import pygame  # noqa
    print('Dependencies already installed.')
except Exception as e:
    print('Installing numpy + pygame ...')
    pip_install(['numpy', 'pygame'])
    print('Done. Re-run this cell if needed, then continue.')


Installing numpy + pygame ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 215.3 kB/s  0:01:19m0:00:0100:03
Done. Re-run this cell if needed, then continue.


## 1) PuzzleState


In [2]:
from __future__ import annotations

from dataclasses import dataclass
from typing import List, Optional, Tuple

BOARD_LEN = 9
N = 3  # 3x3


@dataclass(frozen=True)
class PuzzleState:
    board: Tuple[int, ...]
    parent: Optional['PuzzleState'] = None
    move: Optional[str] = None   # 'up', 'down', 'left', 'right' (movement of the empty tile)
    depth: int = 0

    def __post_init__(self):
        if len(self.board) != BOARD_LEN:
            raise ValueError(f'Board must have length {BOARD_LEN}. Got {len(self.board)}.')
        if set(self.board) != set(range(BOARD_LEN)):
            raise ValueError('Board must contain exactly the numbers 0..8 once each.')

    @staticmethod
    def from_list(board_list: List[int], parent=None, move=None, depth=0) -> 'PuzzleState':
        return PuzzleState(tuple(board_list), parent=parent, move=move, depth=depth)

    def find_zero(self) -> int:
        return self.board.index(0)

    def successor(self) -> List['PuzzleState']:
        """Generate next states by moving the empty tile (0)."""
        z = self.find_zero()
        r, c = divmod(z, N)

        def make_child(new_r: int, new_c: int, move_name: str) -> 'PuzzleState':
            new_idx = new_r * N + new_c
            b = list(self.board)
            b[z], b[new_idx] = b[new_idx], b[z]
            return PuzzleState(tuple(b), parent=self, move=move_name, depth=self.depth + 1)

        children: List[PuzzleState] = []

        if r > 0:
            children.append(make_child(r - 1, c, 'up'))
        if r < N - 1:
            children.append(make_child(r + 1, c, 'down'))
        if c > 0:
            children.append(make_child(r, c - 1, 'left'))
        if c < N - 1:
            children.append(make_child(r, c + 1, 'right'))

        return children


## 2) BFS / DFS Solver


In [3]:
from dataclasses import dataclass
from collections import deque
from typing import Optional, List, Tuple


@dataclass
class SearchResult:
    path: List[PuzzleState]      # from start to goal inclusive
    moves: List[str]            # move names (len = len(path)-1)
    nodes_expanded: int
    visited_count: int
    frontier_max: int


class AgentSolver:
    def __init__(self, start_state: PuzzleState, goal_state: PuzzleState):
        self.start = start_state
        self.goal = goal_state

    def is_goal(self, state: PuzzleState) -> bool:
        return state.board == self.goal.board

    def reconstruct_path(self, goal_state: PuzzleState) -> Tuple[List[PuzzleState], List[str]]:
        states: List[PuzzleState] = []
        moves: List[str] = []
        cur: Optional[PuzzleState] = goal_state
        while cur is not None:
            states.append(cur)
            if cur.move is not None:
                moves.append(cur.move)
            cur = cur.parent
        states.reverse()
        moves.reverse()
        return states, moves

    def bfs(self, max_expansions: Optional[int] = None) -> Optional[SearchResult]:
        frontier = deque([self.start])
        visited = {self.start.board}
        nodes_expanded = 0
        frontier_max = 1

        while frontier:
            state = frontier.popleft()

            if self.is_goal(state):
                path, moves = self.reconstruct_path(state)
                return SearchResult(path, moves, nodes_expanded, len(visited), frontier_max)

            nodes_expanded += 1
            if max_expansions is not None and nodes_expanded > max_expansions:
                return None

            for child in state.successor():
                if child.board not in visited:
                    visited.add(child.board)
                    frontier.append(child)
                    if len(frontier) > frontier_max:
                        frontier_max = len(frontier)

        return None

    def dfs(self, max_expansions: Optional[int] = None) -> Optional[SearchResult]:
        frontier = [self.start]  # stack
        visited = {self.start.board}
        nodes_expanded = 0
        frontier_max = 1

        while frontier:
            state = frontier.pop()

            if self.is_goal(state):
                path, moves = self.reconstruct_path(state)
                return SearchResult(path, moves, nodes_expanded, len(visited), frontier_max)

            nodes_expanded += 1
            if max_expansions is not None and nodes_expanded > max_expansions:
                return None

            children = state.successor()
            for child in reversed(children):
                if child.board not in visited:
                    visited.add(child.board)
                    frontier.append(child)
                    if len(frontier) > frontier_max:
                        frontier_max = len(frontier)

        return None


## 3) Quick test (CLI) — verify BFS works before GUI


In [4]:
# A simple test where the puzzle is 2 moves away from goal.

start = PuzzleState((1,2,3,4,5,6,7,0,8))
goal  = PuzzleState((1,2,3,4,5,6,7,8,0))
solver = AgentSolver(start, goal)
res = solver.bfs(max_expansions=10000)

if res is None:
    print('No solution found (unexpected for this test).')
else:
    print('Moves:', res.moves)
    print('Depth:', len(res.moves))
    print('Expanded:', res.nodes_expanded, 'Visited:', res.visited_count)


Moves: ['right']
Depth: 1
Expanded: 3 Visited: 8


## 4) Pygame GUI (modified to solve + animate)

Important: this opens a **separate window** (typical on your local machine).


In [5]:
import random
import sys
from time import sleep
from typing import List

import pygame


def is_solvable(board_1d: List[int]) -> bool:
    # For 3x3: solvable iff inversion count is even (ignoring 0).
    arr = [x for x in board_1d if x != 0]
    inv = 0
    for i in range(len(arr)):
        for j in range(i + 1, len(arr)):
            if arr[i] > arr[j]:
                inv += 1
    return (inv % 2) == 0


class EightPuzzleGUI:
    def __init__(self, tile_size=110, grid_size=3):
        pygame.init()

        self.TILE_SIZE = tile_size
        self.GRID_SIZE = grid_size
        self.WIDTH = tile_size * grid_size
        self.INFO_H = 80
        self.HEIGHT = tile_size * grid_size + self.INFO_H

        self.WHITE = (255, 255, 255)
        self.BLACK = (0, 0, 0)
        self.BLUE = (70, 130, 180)
        self.GRAY = (240, 240, 240)

        self.FONT = pygame.font.SysFont('Arial', 40, bold=True)
        self.INFO_FONT = pygame.font.SysFont('Arial', 18, bold=False)

        self.screen = pygame.display.set_mode((self.WIDTH, self.HEIGHT))
        pygame.display.set_caption('8 Puzzle (BFS/DFS Solver)')

        self.goal_1d = [1, 2, 3, 4, 5, 6, 7, 8, 0]
        self.grid = self.create_puzzle()

        self.last_algo = '-'
        self.last_moves = 0
        self.last_expanded = 0
        self.message = 'Arrows: move | B: BFS | D: DFS | R: reshuffle | G: goal'

    def grid_to_1d(self) -> List[int]:
        out = []
        for r in range(self.GRID_SIZE):
            for c in range(self.GRID_SIZE):
                out.append(self.grid[r][c])
        return out

    def apply_1d(self, board_1d: List[int]) -> None:
        self.grid = [
            board_1d[i:i + self.GRID_SIZE]
            for i in range(0, self.GRID_SIZE * self.GRID_SIZE, self.GRID_SIZE)
        ]

    def create_puzzle(self) -> List[List[int]]:
        nums = list(range(1, 9)) + [0]
        while True:
            random.shuffle(nums)
            if nums != self.goal_1d and is_solvable(nums):
                break
        return [nums[i:i + self.GRID_SIZE] for i in range(0, 9, self.GRID_SIZE)]

    def find_empty(self):
        for r in range(self.GRID_SIZE):
            for c in range(self.GRID_SIZE):
                if self.grid[r][c] == 0:
                    return r, c
        return None

    def move(self, direction: str) -> None:
        r, c = self.find_empty()

        if direction == 'up' and r > 0:
            self.grid[r][c], self.grid[r - 1][c] = self.grid[r - 1][c], self.grid[r][c]
        elif direction == 'down' and r < self.GRID_SIZE - 1:
            self.grid[r][c], self.grid[r + 1][c] = self.grid[r + 1][c], self.grid[r][c]
        elif direction == 'left' and c > 0:
            self.grid[r][c], self.grid[r][c - 1] = self.grid[r][c - 1], self.grid[r][c]
        elif direction == 'right' and c < self.GRID_SIZE - 1:
            self.grid[r][c], self.grid[r][c + 1] = self.grid[r][c + 1], self.grid[r][c]

    def draw(self) -> None:
        self.screen.fill(self.WHITE)

        for r in range(self.GRID_SIZE):
            for c in range(self.GRID_SIZE):
                val = self.grid[r][c]
                rect = pygame.Rect(c * self.TILE_SIZE, r * self.TILE_SIZE, self.TILE_SIZE, self.TILE_SIZE)

                pygame.draw.rect(self.screen, self.BLUE if val != 0 else self.WHITE, rect)
                pygame.draw.rect(self.screen, self.BLACK, rect, 2)

                if val != 0:
                    text = self.FONT.render(str(val), True, self.WHITE)
                    self.screen.blit(text, text.get_rect(center=rect.center))

        info_rect = pygame.Rect(0, self.WIDTH, self.WIDTH, self.INFO_H)
        pygame.draw.rect(self.screen, self.GRAY, info_rect)
        pygame.draw.rect(self.screen, self.BLACK, info_rect, 1)

        lines = [
            self.message,
            f'Last solve: {self.last_algo} | moves={self.last_moves} | expanded={self.last_expanded}',
        ]
        y = self.WIDTH + 10
        for line in lines:
            t = self.INFO_FONT.render(line, True, self.BLACK)
            self.screen.blit(t, (10, y))
            y += 24

        pygame.display.flip()

    def animate_solution(self, path_states: List[PuzzleState], delay_s: float = 0.25) -> None:
        for st in path_states:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

            self.apply_1d(list(st.board))
            self.draw()
            sleep(delay_s)

    def solve_current(self, algo: str) -> None:
        start_1d = self.grid_to_1d()
        start = PuzzleState(tuple(start_1d))
        goal = PuzzleState(tuple(self.goal_1d))
        solver = AgentSolver(start, goal)

        self.last_algo = algo.upper()
        self.message = f'Solving with {self.last_algo}...'
        self.draw()
        pygame.event.pump()

        if algo == 'bfs':
            result = solver.bfs(max_expansions=250_000)
        else:
            result = solver.dfs(max_expansions=250_000)

        if result is None:
            self.last_moves = 0
            self.last_expanded = 0
            self.message = f'{self.last_algo} failed (limit reached). Press R to reshuffle.'
            return

        self.last_moves = len(result.moves)
        self.last_expanded = result.nodes_expanded
        self.message = f'{self.last_algo} solution found. Animating...'

        self.animate_solution(result.path, delay_s=0.25)
        self.message = 'Done. Arrows: move | B: BFS | D: DFS | R: reshuffle | G: goal'

    def run(self) -> None:
        running = True
        while running:
            self.draw()
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        self.move('up')
                    elif event.key == pygame.K_DOWN:
                        self.move('down')
                    elif event.key == pygame.K_LEFT:
                        self.move('left')
                    elif event.key == pygame.K_RIGHT:
                        self.move('right')
                    elif event.key == pygame.K_r:
                        self.grid = self.create_puzzle()
                        self.last_algo, self.last_moves, self.last_expanded = '-', 0, 0
                        self.message = 'Reshuffled. Press B (BFS) or D (DFS).'
                    elif event.key == pygame.K_g:
                        self.apply_1d(self.goal_1d)
                        self.message = 'Set to goal state.'
                    elif event.key == pygame.K_b:
                        self.solve_current('bfs')
                    elif event.key == pygame.K_d:
                        self.solve_current('dfs')

        pygame.quit()
        sys.exit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


## 5) Run the GUI

Run the next cell to open the game window.


In [8]:
gui = EightPuzzleGUI(tile_size=110, grid_size=3)
gui.run()


SystemExit: 